# #0 Table of Content
1. Import packages and data
2. Data correction
   2.1. Merchant Table
   2.2. Product Table
   2.3. Review Table
   2.4. User Table 
4. Export CSV

# #1 Import packages, data and creating functions

In [1]:
import pandas as pd
import uuid

In [2]:
merchant_table = pd.read_csv("../asset/merchant.csv")
product_table  = pd.read_csv("../asset/product.csv")
review_table   = pd.read_csv("../asset/review.csv")

In [3]:
def prefix_fix(x):
    if 'k' in x or 'K' in x:
        return int(float(x[:-1]) * 1000)
        
    elif 'm' in x or 'M' in x:
        return int(float(x[:-1]) * 1000000)

    else:
        return int(x)

def product_totalSold_fix(x):
    if 'k' in x or 'K' in x:
        return int(float(x[:-1]) * 1000)
        
    elif 'm' in x or 'M' in x:
        return int(float(x[:-1]) * 1000000)

    else:
        return int(x)

def product_price_fix(x):
    if '-' in x:
        x = x.split(' - ')[0]

    x = x.replace(',','').strip()[1:]
    
    return float(x)

def product_qtyAvail_fix(x):
    x = x.split(' ')[0]
    return int(x)

def product_favCount_fix(x):
    x = x.split(' ')[-1]

    if x == '0':
        return int(x)
    else:
        x = x[1:-1]
        return prefix_fix(x)

def review_location_fix(x):
    if ':' not in x:
        return x

def review_date_fix(x):
    if ':' in x:
        return pd.to_datetime(x, format="%d/%m/%Y %H:%M")
    else:
        return None

def merchant_responseRatePercent_fix(x):
    return float(x[:-1])

def merchant_days_fix(x):
    num  = x.split(' ')[0]
    unit = x.split(' ')[1]

    match unit:
        case "years":
            return int(num) * 365
        case "months":
            return int(num) * 30
        case "weeks":
            return int(num) * 7
        case "days":
            return int(num)
        case _:
            return None


# #2 Data Correction

## #2.1 Merchant Table

In [4]:
# MERCHANT
print(merchant_table.shape)
display(merchant_table.head(3))
print(merchant_table.info())

(592, 8)


,merchant_id,name,Ratings,response rate,joined,products,response time,follower
0,983b8576-0dd8-4c84-acdf-0915734adbd0,sprise_localstore.sg,809,96%,7 months ago,12,within hours,630
1,6e3b6c48-c873-44ac-b99a-845f35a30cf7,IN-BOX,24.2k,97%,8 years ago,366,within hours,11.5k
2,90f139bf-4bb4-4f5c-8511-f99ede1d71b2,Edifier Flagship Store,367,97%,7 months ago,24,within hours,2.5k


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   merchant_id    592 non-null    object
 1   name           592 non-null    object
 2   Ratings        592 non-null    object
 3   response rate  592 non-null    object
 4   joined         592 non-null    object
 5   products       592 non-null    object
 6   response time  592 non-null    object
 7   follower       592 non-null    object
dtypes: object(8)
memory usage: 37.1+ KB
None


In [5]:
def first_id(x):
    return x[0]

merchant_id_map_df = merchant_table[merchant_table.name.duplicated(keep=False)].groupby('name')['merchant_id'].agg(list).reset_index()
merchant_id_map_df = merchant_id_map_df.rename(columns={'merchant_id':'merchant_id_list'})
merchant_id_map_df['merchant_id'] = merchant_id_map_df['merchant_id_list'].apply(first_id)
merchant_id_map_df

,name,merchant_id_list,merchant_id
0,12BUY.SG,"[5fbad251-5c05-4f84-87d5-06c5a38a4308, c87e3ed...",5fbad251-5c05-4f84-87d5-06c5a38a4308
1,32DayTech,"[8c98f3b0-0e36-4c77-a003-24f2ffe151ca, 9a9bda2...",8c98f3b0-0e36-4c77-a003-24f2ffe151ca
2,ALLSTARS,"[fdde3e35-25fd-49b2-8e65-7a0a7893557e, 9512454...",fdde3e35-25fd-49b2-8e65-7a0a7893557e
3,ASUS Official Store,"[b9a90c02-41a9-40ac-b003-47a76ab50396, b66979f...",b9a90c02-41a9-40ac-b003-47a76ab50396
4,Acer Official Store,"[50ab6104-b628-4ff1-b6c9-ddc4a296d07a, 74819bb...",50ab6104-b628-4ff1-b6c9-ddc4a296d07a
...,...,...,...
98,samanthayap1983,"[ea995824-634e-4cca-8b82-f25a239bbaaa, cac5670...",ea995824-634e-4cca-8b82-f25a239bbaaa
99,spoetry01.sg,"[6a69cc5f-8a17-4b2f-92b3-c28f08c8dc03, 0b0358b...",6a69cc5f-8a17-4b2f-92b3-c28f08c8dc03
100,sprise_localstore.sg,"[983b8576-0dd8-4c84-acdf-0915734adbd0, 39d81d0...",983b8576-0dd8-4c84-acdf-0915734adbd0
101,wowking｜Life Technology Digital,"[833aa515-5578-42d7-a143-bb426efeaa4e, 7ddd041...",833aa515-5578-42d7-a143-bb426efeaa4e


In [6]:
merchant_id_map_list = [(row['merchant_id_list'], row['merchant_id']) for index, row in merchant_id_map_df.iterrows()]
merchant_id_map_list

[(['5fbad251-5c05-4f84-87d5-06c5a38a4308',
   'c87e3edb-b4ae-456a-83ad-bd42d2d316ce'],
  '5fbad251-5c05-4f84-87d5-06c5a38a4308'),
 (['8c98f3b0-0e36-4c77-a003-24f2ffe151ca',
   '9a9bda2c-8459-48d5-923b-e3663dfd70d2'],
  '8c98f3b0-0e36-4c77-a003-24f2ffe151ca'),
 (['fdde3e35-25fd-49b2-8e65-7a0a7893557e',
   '95124541-431e-464a-86d0-9fbda7a98f4a'],
  'fdde3e35-25fd-49b2-8e65-7a0a7893557e'),
 (['b9a90c02-41a9-40ac-b003-47a76ab50396',
   'b66979fb-8bcb-43ce-9d9b-c00a01e843f5',
   '11c705ef-e672-43cc-8041-ab246c9d52bc',
   'ca7cd5c1-8a97-4ffb-8518-3d449fad6d64'],
  'b9a90c02-41a9-40ac-b003-47a76ab50396'),
 (['50ab6104-b628-4ff1-b6c9-ddc4a296d07a',
   '74819bbd-ab3f-48d0-98f3-cee8294c73e8',
   'd6e1bb23-7ad2-4794-b820-8ab04e74a94c',
   'da1713a7-4bd0-4aad-8fbc-1ef3060e2f09',
   'd7878400-dcfd-44b0-9a5f-eec8fa430c92',
   'e838f7af-3e75-4d85-9a77-2e2c1ec09f4c',
   '428ca980-fa37-4ef1-8772-a728d457fea4',
   'd4e3e458-163f-4777-bea7-0f09906533b2',
   '385c8eae-489f-42d2-a38a-a087cb66d04e',
   '80e

In [7]:
merchant_id_map = {}
for _tuple in merchant_id_map_list:
    for _id in _tuple[0]:
        merchant_id_map[_id] = _tuple[1]

merchant_id_map

{'5fbad251-5c05-4f84-87d5-06c5a38a4308': '5fbad251-5c05-4f84-87d5-06c5a38a4308',
 'c87e3edb-b4ae-456a-83ad-bd42d2d316ce': '5fbad251-5c05-4f84-87d5-06c5a38a4308',
 '8c98f3b0-0e36-4c77-a003-24f2ffe151ca': '8c98f3b0-0e36-4c77-a003-24f2ffe151ca',
 '9a9bda2c-8459-48d5-923b-e3663dfd70d2': '8c98f3b0-0e36-4c77-a003-24f2ffe151ca',
 'fdde3e35-25fd-49b2-8e65-7a0a7893557e': 'fdde3e35-25fd-49b2-8e65-7a0a7893557e',
 '95124541-431e-464a-86d0-9fbda7a98f4a': 'fdde3e35-25fd-49b2-8e65-7a0a7893557e',
 'b9a90c02-41a9-40ac-b003-47a76ab50396': 'b9a90c02-41a9-40ac-b003-47a76ab50396',
 'b66979fb-8bcb-43ce-9d9b-c00a01e843f5': 'b9a90c02-41a9-40ac-b003-47a76ab50396',
 '11c705ef-e672-43cc-8041-ab246c9d52bc': 'b9a90c02-41a9-40ac-b003-47a76ab50396',
 'ca7cd5c1-8a97-4ffb-8518-3d449fad6d64': 'b9a90c02-41a9-40ac-b003-47a76ab50396',
 '50ab6104-b628-4ff1-b6c9-ddc4a296d07a': '50ab6104-b628-4ff1-b6c9-ddc4a296d07a',
 '74819bbd-ab3f-48d0-98f3-cee8294c73e8': '50ab6104-b628-4ff1-b6c9-ddc4a296d07a',
 'd6e1bb23-7ad2-4794-b820-8a

In [8]:
merchant_table = merchant_table.drop_duplicates(subset=['name'])

merchant_table['merchant_id'] = merchant_table['merchant_id'].map(merchant_id_map).fillna(merchant_table['merchant_id'])

In [9]:
rename_dict = {
    'Ratings'       : 'total_rating',
    'response rate' : 'response_rate_percent',
    'joined'        : 'days',
    'products'      : 'no_products',
    'response time': 'response_speed',
    'follower'      : 'no_follower'
}

merchant_table = merchant_table.rename(columns=rename_dict)

In [10]:
if merchant_table['total_rating'].dtype != 'int64':
    merchant_table['total_rating'] = merchant_table['total_rating'].apply(prefix_fix)

if merchant_table['response_rate_percent'].dtype != 'float64':
    merchant_table['response_rate_percent'] = merchant_table['response_rate_percent'].apply(merchant_responseRatePercent_fix)

if merchant_table['days'].dtype != 'int64':
    merchant_table['days'] = merchant_table['days'].apply(merchant_days_fix)

if merchant_table['no_products'].dtype != 'int64':
    merchant_table['no_products'] = merchant_table['no_products'].apply(prefix_fix)

if merchant_table['no_follower'].dtype != 'int64':
    merchant_table['no_follower'] = merchant_table['no_follower'].apply(prefix_fix)

print('total_rating          : ', merchant_table['total_rating'].dtype)
print('response_rate_percent : ', merchant_table['response_rate_percent'].dtype)
print('days                  : ', merchant_table['days'].dtype)
print('no_products           : ', merchant_table['no_products'].dtype)
print('no_follower           : ', merchant_table['no_follower'].dtype)

print(merchant_table.shape)
display(merchant_table.head(3))
print(merchant_table.info())

total_rating          :  int64
response_rate_percent :  float64
days                  :  int64
no_products           :  int64
no_follower           :  int64
(290, 8)


,merchant_id,name,total_rating,response_rate_percent,days,no_products,response_speed,no_follower
0,983b8576-0dd8-4c84-acdf-0915734adbd0,sprise_localstore.sg,809,96.0,210,12,within hours,630
1,6e3b6c48-c873-44ac-b99a-845f35a30cf7,IN-BOX,24200,97.0,2920,366,within hours,11500
2,90f139bf-4bb4-4f5c-8511-f99ede1d71b2,Edifier Flagship Store,367,97.0,210,24,within hours,2500


<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, 0 to 590
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   merchant_id            290 non-null    object 
 1   name                   290 non-null    object 
 2   total_rating           290 non-null    int64  
 3   response_rate_percent  290 non-null    float64
 4   days                   290 non-null    int64  
 5   no_products            290 non-null    int64  
 6   response_speed         290 non-null    object 
 7   no_follower            290 non-null    int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 20.4+ KB
None


## #2.2 Product Table

In [11]:
# PRODUCT
print(product_table.shape)
display(product_table.head(3))
print(product_table.info())

(592, 14)


,product_id,category,merchant_id,name,preferred,mall,avg_rating,total_rating,total_sold,price,qty_avail,fav_count,img_src,description
0,912d838b-5e84-4aef-a0dd-bb23f44e5913,headphone,983b8576-0dd8-4c84-acdf-0915734adbd0,SPRISE Premium Wireless Bluetooth Earphone Col...,1,0,4.6,256,840,$17.39 - $18.78,459 pieces available,Favorite (1.1k),https://down-sg.img.susercontent.com/file/my-1...,Hey~ Welcome to SPRISE Official Store! Please ...
1,34bb8bfe-e134-41af-a487-3b8d4ce6fff0,headphone,6e3b6c48-c873-44ac-b99a-845f35a30cf7,Rock Space [SG] O2 Wireless Headphone Bluetoot...,1,0,4.9,369,955,$35.80,164 pieces available,Favorite (1.8k),https://down-sg.img.susercontent.com/file/sg-1...,SKU: 2856 Name: Rock Space 02 Wireless Headpho...
2,35ff0f3c-8a06-4e33-a789-319ff793220d,headphone,90f139bf-4bb4-4f5c-8511-f99ede1d71b2,Edifier W820NB/W820NB PLUS Wireless Headphone ...,1,0,4.8,88,259,$79.99 - $99.00,329 pieces available,Favorite (466),https://down-sg.img.susercontent.com/file/cn-1...,EDIFIER W820NB PLUS NOISE CANCELING ACTIVE NOI...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    592 non-null    object 
 1   category      592 non-null    object 
 2   merchant_id   592 non-null    object 
 3   name          592 non-null    object 
 4   preferred     592 non-null    int64  
 5   mall          592 non-null    int64  
 6   avg_rating    570 non-null    float64
 7   total_rating  570 non-null    object 
 8   total_sold    592 non-null    object 
 9   price         592 non-null    object 
 10  qty_avail     591 non-null    object 
 11  fav_count     592 non-null    object 
 12  img_src       170 non-null    object 
 13  description   584 non-null    object 
dtypes: float64(1), int64(2), object(11)
memory usage: 64.9+ KB
None


In [12]:
product_table['merchant_id'] = product_table['merchant_id'].map(merchant_id_map).fillna(product_table['merchant_id'])

In [13]:
if 'preferred' in product_table.columns:
    product_table = product_table.drop('preferred', axis=1)

if 'mall' in product_table.columns:
    product_table = product_table.drop('mall', axis=1)

product_table['avg_rating'] = product_table['avg_rating'].fillna(0)

if product_table['total_rating'].dtype != 'int64':
    product_table['total_rating'] = product_table['total_rating'].fillna('0')
    product_table['total_rating'] = product_table['total_rating'].apply(prefix_fix)

if product_table['total_sold'].dtype != 'int64':
    product_table['total_sold'] = product_table['total_sold'].apply(prefix_fix)

if product_table['price'].dtype != 'float64':
    product_table['price'] = product_table['price'].apply(product_price_fix)

if product_table['qty_avail'].dtype != 'int64':
    product_table['qty_avail'] = product_table['qty_avail'].fillna('0')
    product_table['qty_avail'] = product_table['qty_avail'].apply(product_qtyAvail_fix)

if product_table['fav_count'].dtype != 'int64':
    product_table['fav_count'] = product_table['fav_count'].apply(product_favCount_fix)

print('total_rating  :', product_table['total_rating'].dtype)
print('total_sold    :', product_table['total_sold'].dtype)
print('price         :', product_table['price'].dtype)
print('qty_avail     :', product_table['qty_avail'].dtype)
print('fav_count     :', product_table['fav_count'].dtype)
display(product_table[['total_sold','total_rating','price','qty_avail','fav_count']].head(3))

print(product_table.shape)
display(product_table.head(3))
print(product_table.info())

total_rating  : int64
total_sold    : int64
price         : float64
qty_avail     : int64
fav_count     : int64


,total_sold,total_rating,price,qty_avail,fav_count
0,840,256,17.39,459,1100
1,955,369,35.80,164,1800
2,259,88,79.99,329,466


(592, 12)


,product_id,category,merchant_id,name,avg_rating,total_rating,total_sold,price,qty_avail,fav_count,img_src,description
0,912d838b-5e84-4aef-a0dd-bb23f44e5913,headphone,983b8576-0dd8-4c84-acdf-0915734adbd0,SPRISE Premium Wireless Bluetooth Earphone Col...,4.6,256,840,17.39,459,1100,https://down-sg.img.susercontent.com/file/my-1...,Hey~ Welcome to SPRISE Official Store! Please ...
1,34bb8bfe-e134-41af-a487-3b8d4ce6fff0,headphone,6e3b6c48-c873-44ac-b99a-845f35a30cf7,Rock Space [SG] O2 Wireless Headphone Bluetoot...,4.9,369,955,35.80,164,1800,https://down-sg.img.susercontent.com/file/sg-1...,SKU: 2856 Name: Rock Space 02 Wireless Headpho...
2,35ff0f3c-8a06-4e33-a789-319ff793220d,headphone,90f139bf-4bb4-4f5c-8511-f99ede1d71b2,Edifier W820NB/W820NB PLUS Wireless Headphone ...,4.8,88,259,79.99,329,466,https://down-sg.img.susercontent.com/file/cn-1...,EDIFIER W820NB PLUS NOISE CANCELING ACTIVE NOI...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    592 non-null    object 
 1   category      592 non-null    object 
 2   merchant_id   592 non-null    object 
 3   name          592 non-null    object 
 4   avg_rating    592 non-null    float64
 5   total_rating  592 non-null    int64  
 6   total_sold    592 non-null    int64  
 7   price         592 non-null    float64
 8   qty_avail     592 non-null    int64  
 9   fav_count     592 non-null    int64  
 10  img_src       170 non-null    object 
 11  description   584 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 55.6+ KB
None


## #2.3 Review Table

In [14]:
# REVIEW
print(review_table.shape)
display(review_table.head(3))
print(review_table.info())

(155253, 7)


,review_id,username,merchant_id,product_id,date,rating,content
0,6ef347c0-c603-422a-b8d1-c4b96bed0207,i*****b,983b8576-0dd8-4c84-acdf-0915734adbd0,912d838b-5e84-4aef-a0dd-bb23f44e5913,2/9/2023 1:13,5,Best buy ever\r\nit looks great works great\r\...
1,af3ccbed-1865-4492-88e3-723e9dda0de9,jessylim70,983b8576-0dd8-4c84-acdf-0915734adbd0,912d838b-5e84-4aef-a0dd-bb23f44e5913,21/7/2023 23:33,5,Item received in good condition.\r\nBought dur...
2,d17ebe72-919d-4c2a-a230-88119aac725c,s*****b,983b8576-0dd8-4c84-acdf-0915734adbd0,912d838b-5e84-4aef-a0dd-bb23f44e5913,8/5/2023 19:37,5,Value For Money: yes\r\nBest Feature(s): comfo...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155253 entries, 0 to 155252
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review_id    155253 non-null  object
 1   username     155253 non-null  object
 2   merchant_id  155253 non-null  object
 3   product_id   155253 non-null  object
 4   date         155253 non-null  object
 5   rating       155253 non-null  int64 
 6   content      81997 non-null   object
dtypes: int64(1), object(6)
memory usage: 8.3+ MB
None


In [15]:
if 'merchant_id' in review_table.columns:
    review_table = review_table.drop('merchant_id', axis=1)

review_table['location'] = review_table['date'].apply(review_location_fix)
review_table['date'] = review_table['date'].apply(review_date_fix)

In [16]:
print(review_table.shape)
display(review_table.head(3))
print(review_table.info())

(155253, 7)


,review_id,username,product_id,date,rating,content,location
0,6ef347c0-c603-422a-b8d1-c4b96bed0207,i*****b,912d838b-5e84-4aef-a0dd-bb23f44e5913,2023-09-02 01:13:00,5,Best buy ever\r\nit looks great works great\r\...,None
1,af3ccbed-1865-4492-88e3-723e9dda0de9,jessylim70,912d838b-5e84-4aef-a0dd-bb23f44e5913,2023-07-21 23:33:00,5,Item received in good condition.\r\nBought dur...,None
2,d17ebe72-919d-4c2a-a230-88119aac725c,s*****b,912d838b-5e84-4aef-a0dd-bb23f44e5913,2023-05-08 19:37:00,5,Value For Money: yes\r\nBest Feature(s): comfo...,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155253 entries, 0 to 155252
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   review_id   155253 non-null  object        
 1   username    155253 non-null  object        
 2   product_id  155253 non-null  object        
 3   date        141225 non-null  datetime64[ns]
 4   rating      155253 non-null  int64         
 5   content     81997 non-null   object        
 6   location    14028 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 8.3+ MB
None


## #2.4 User Table

In [17]:
aggregate = {
    "review_id"  : 'nunique',
    "product_id" : 'nunique'
}

rename_dict = {
    'review_id' :"no_review",
    'product_id':"no_product"
}

user_table = review_table.groupby('username').agg(aggregate).reset_index()
user_table = user_table.rename(columns=rename_dict)

display(user_table.head(3))

,username,no_review,no_product
0,.*****.,10,5
1,.*****9,1,1
2,.*****_,2,2


In [18]:
review_list_table = review_table.groupby('username')['review_id'].agg(list).reset_index()
display(review_list_table.head(3))

,username,review_id
0,.*****.,"[03afe402-5369-42bf-9b01-1427b239d0ee, afa6c7a..."
1,.*****9,[a0687dbd-7bf2-47a9-b3c3-d65be8c07cda]
2,.*****_,"[7a2df4e5-6822-4e7a-836b-4b315382a643, aaef11f..."


In [19]:
def product_dict_fix(x):
    product_dict = {}
    for item in x:
        if item not in product_dict:
            product_dict[item] = 1
        else:
            product_dict[item] += 1
    return product_dict

product_dict_table = review_table.groupby('username')['product_id'].agg(list).reset_index()
product_dict_table['product_id'] = product_dict_table['product_id'].apply(product_dict_fix)
display(product_dict_table.head(3))

,username,product_id
0,.*****.,"{'d19cf120-93e6-4f1c-89c2-2b18187139cb': 2, '5..."
1,.*****9,{'188cb6f8-9a76-4c58-aab3-1dbc11cb2b2e': 1}
2,.*****_,"{'cbd4fe9d-1474-4653-9e63-9d8bd9233413': 1, '8..."


In [20]:
user_table = user_table.merge(review_list_table, on='username', how='inner').merge(product_dict_table, on='username', how='inner')
display(user_table.head(3))

,username,no_review,no_product,review_id,product_id
0,.*****.,10,5,"[03afe402-5369-42bf-9b01-1427b239d0ee, afa6c7a...","{'d19cf120-93e6-4f1c-89c2-2b18187139cb': 2, '5..."
1,.*****9,1,1,[a0687dbd-7bf2-47a9-b3c3-d65be8c07cda],{'188cb6f8-9a76-4c58-aab3-1dbc11cb2b2e': 1}
2,.*****_,2,2,"[7a2df4e5-6822-4e7a-836b-4b315382a643, aaef11f...","{'cbd4fe9d-1474-4653-9e63-9d8bd9233413': 1, '8..."


# #3 Export CSV

In [21]:
merchant_table.to_csv("../asset/etlMerchant.csv", index=False)
product_table.to_csv("../asset/etlProduct.csv", index=False)
review_table.to_csv("../asset/etlReview.csv", index=False)
user_table.to_csv("../asset/etlUser.csv", index=False)

PermissionError: [Errno 13] Permission denied: '../asset/etlReview.csv'